<a href="https://colab.research.google.com/github/Blutronics/OffensiveTwitterDetection/blob/Toan/OffensiveDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1260]:
import numpy as np
import pandas as pd
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,r2_score
import xgboost as xgb
import lightgbm as gbm

In [1261]:
data=pd.read_csv("twitter.csv")
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [1262]:
classdf=data[['class','tweet']]
classdf.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [1263]:
tweets=classdf.tweet.to_string()

In [1264]:
tweets = re.sub("[^A-Za-z" "]+"," ", tweets).lower()
tweets = re.sub("[0-9" "]+"," ", tweets)

In [1265]:
class2df=classdf.loc[classdf['class']==2]
class2df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
40,2,""" momma said no pussy cats inside my doghouse """
63,2,"""@Addicted2Guys: -SimplyAddictedToGuys http://..."
66,2,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo..."
67,2,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these..."


In [1266]:
offensivedf=classdf.loc[classdf['class']==1]
offensivedf.head()

,class,tweet
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."


In [1267]:
hatedf=classdf.loc[classdf['class']==0]
hatedf.head()

,class,tweet
85,0,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1..."
89,0,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a ..."
110,0,"""@DevilGrimz: @VigxRArts you're fucking gay, b..."
184,0,"""@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL..."
202,0,"""@NoChillPaz: ""At least I'm not a nigger"" http..."


In [1268]:
offensivetweets=offensivedf.tweet.to_string()


In [1269]:
offensivetweets = re.sub("[^A-Za-z" "]+"," ", offensivetweets).lower()
offensivetweets = re.sub("[0-9" "]+"," ", offensivetweets)

In [1270]:
offensivetweets = offensivetweets.split(" ")

In [1271]:
hatespeachtweets=hatedf.tweet.to_string()


In [1272]:
hatespeachtweets = re.sub("[^A-Za-z" "]+"," ", hatespeachtweets).lower()
hatespeachtweets = re.sub("[0-9" "]+"," ", hatespeachtweets)

In [1273]:
hatespeachtweets = hatespeachtweets.split(" ")

In [1274]:
#hatespeachtweets

In [1275]:
vectorizer = TfidfVectorizer()
offensive = vectorizer.fit_transform(offensivetweets)

In [1276]:
with open("stop.txt","r") as sw:
    stop_words = sw.read()

In [1277]:
stop_words = stop_words.split("\n")

In [1278]:
stop_words.extend(["rt","a"])

In [1279]:
offensivetweets = [w for w in offensivetweets if not w in stop_words]


In [1280]:
offensivetweets = " ".join(offensivetweets)

In [1281]:
'''
wordcloud_offensive = WordCloud(
                      background_color='White',
                      width=1800,
                      height=1400
                     ).generate(offensivetweets)'''

"\nwordcloud_offensive = WordCloud(\n                      background_color='White',\n                      width=1800,\n                      height=1400\n                     ).generate(offensivetweets)"

In [1282]:
#plt.imshow(wordcloud_offensive)

In [1283]:
offensivetweets = offensivetweets.split(" ")

In [1284]:
letter_counts = Counter(offensivetweets)
letter_counts.pop('')


1

In [1285]:
most_offensive = letter_counts.most_common(25)
most_offensive

[('bitch', 4601),
 ('bitches', 1784),
 ('hoes', 1377),
 ('pussy', 1059),
 ('hoe', 1027),
 ('don', 714),
 ('fuck', 696),
 ('ass', 619),
 ('nigga', 524),
 ('ain', 505),
 ('shit', 440),
 ('niggas', 418),
 ('lol', 407),
 ('yo', 406),
 ('love', 364),
 ('bad', 347),
 ('http', 298),
 ('im', 282),
 ('hate', 282),
 ('ya', 240),
 ('fucking', 236),
 ('amp', 231),
 ('man', 222),
 ('good', 206),
 ('real', 204)]

In [1286]:
hatespeachtweets = [w for w in hatespeachtweets if not w in stop_words]

In [1287]:
letter_counts2 = Counter(hatespeachtweets)
letter_counts2.pop('')

2

In [1288]:
most_hate = letter_counts2.most_common(25)
most_hate

[('faggot', 102),
 ('bitch', 98),
 ('white', 75),
 ('fuck', 73),
 ('nigga', 60),
 ('ass', 54),
 ('trash', 54),
 ('fag', 51),
 ('niggas', 50),
 ('nigger', 48),
 ('fucking', 46),
 ('hate', 43),
 ('don', 39),
 ('people', 34),
 ('faggots', 31),
 ('bitches', 30),
 ('hoes', 29),
 ('http', 28),
 ('hoe', 25),
 ('shit', 23),
 ('fags', 22),
 ('black', 21),
 ('niggers', 20),
 ('stupid', 20),
 ('pussy', 19)]

In [1289]:
def countOccurrences(str, word, listword):
     
    # split the string by spaces in a
    a = str.split(" ")
    # search for pattern in a
    count = 0
    # for i in range(1000):
    for i in range(0, len(a)):
        
        # if match found increase count
        # print(a[i])
      if (word == a[i]):
          count = count + 1
    if count > 0:
      listword.append(1)
    else:
      listword.append(0)
            
    return count, listword

In [1290]:
bitch = []
# for i in range(12):
for i in range(24783):
  count, bitch = countOccurrences(classdf.tweet[i],"bitch", bitch)
  

In [1291]:
bitches = []
# for i in range(12):
for i in range(24783):
  count, bitches = countOccurrences(classdf.tweet[i],"bitches", bitches)
  

In [1292]:
hoes = []
# for i in range(12):
for i in range(24783):
  count, hoes = countOccurrences(classdf.tweet[i],"hoes", hoes)
  

In [1293]:
pussy = []
# for i in range(12):
for i in range(24783):
  count, pussy = countOccurrences(classdf.tweet[i],"pussy", pussy)
  

In [1294]:
hoe = []
# for i in range(12):
for i in range(24783):
  count, hoe = countOccurrences(classdf.tweet[i],"hoe", hoe)
  

In [1295]:
fuck = []
# for i in range(12):
for i in range(24783):
  count, fuck = countOccurrences(classdf.tweet[i],"fuck", fuck)
  

In [1296]:
ass = []
# for i in range(12):
for i in range(24783):
  count, ass = countOccurrences(classdf.tweet[i],"ass", ass)

In [1297]:
nigga = []
# for i in range(12):
for i in range(24783):
  count, nigga = countOccurrences(classdf.tweet[i],"nigga", nigga)

In [1298]:
shit = []
# for i in range(12):
for i in range(24783):
  count, shit = countOccurrences(classdf.tweet[i],"shit", shit)

In [1299]:
niggas = []
# for i in range(12):
for i in range(24783):
  count, niggas = countOccurrences(classdf.tweet[i],"niggas", niggas)

In [1300]:
nigger = []
# for i in range(12):
for i in range(24783):
  count, nigger = countOccurrences(classdf.tweet[i],"nigger", nigger)

In [1301]:
#create a new dataframe counting each frequent word for each tweet(row)
#most frequent word= column

In [1302]:
encodedDf = pd.DataFrame(bitch)

In [1303]:
encodedDf

,0
0,0
1,0
2,1
3,0
4,1
...,...
24778,0
24779,0
24780,0
24781,0


In [1304]:
encodedDf.rename(columns = {0:"Bitch"},inplace=True)

In [1305]:
encodedDf["Bitches"] = bitches

In [1306]:
encodedDf["Hoes"] = hoes

In [1307]:
encodedDf["Pussy"] = pussy

In [1308]:
encodedDf["Hoe"] = hoe

In [1309]:
encodedDf["Fuck"] = fuck

In [1310]:
encodedDf["Ass"] = ass

In [1311]:
encodedDf["Nigga"] = nigga

In [1312]:
encodedDf["Shit"] = shit

In [1313]:
encodedDf["Niggas"] = niggas

In [1314]:
encodedDf["Nigger"] = nigger

In [1315]:
encodedDf

,Bitch,Bitches,Hoes,Pussy,Hoe,Fuck,Ass,Nigga,Shit,Niggas,Nigger
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
24778,0,0,0,0,0,0,0,0,0,0,0
24779,0,0,0,0,0,0,0,0,0,0,0
24780,0,0,0,0,0,0,0,0,0,0,0
24781,0,1,0,0,0,0,0,0,0,0,0


In [1316]:
faggot = []
for i in range(24783):
  count, faggot = countOccurrences(classdf.tweet[i],"faggot", faggot)
  

In [1317]:
bitch = []
for i in range(24783):
  count, bitch = countOccurrences(classdf.tweet[i],"bitch", bitch)

In [1318]:
fuck = []
for i in range(24783):
  count, fuck = countOccurrences(classdf.tweet[i],"fuck", fuck)

In [1319]:
nigga = []
for i in range(24783):
  count, nigga = countOccurrences(classdf.tweet[i],"nigga", nigga)

In [1320]:
ass = []
for i in range(24783):
  count, ass = countOccurrences(classdf.tweet[i],"ass", ass)

In [1321]:
fag = []
for i in range(24783):
  count, fag = countOccurrences(classdf.tweet[i],"fag", fag)

In [1322]:
niggas = []
for i in range(24783):
  count, niggas = countOccurrences(classdf.tweet[i],"niggas", niggas)

In [1323]:
fucking = []
for i in range(24783):
  count, fucking = countOccurrences(classdf.tweet[i],"fucking", fucking)

In [1324]:
faggots = []
for i in range(24783):
  count, faggots = countOccurrences(classdf.tweet[i],"faggots", faggots)

In [1325]:
white = []
for i in range(24783):
  count, white = countOccurrences(classdf.tweet[i],"white", white)

In [1326]:
trash = []
for i in range(24783):
  count, trash = countOccurrences(classdf.tweet[i],"trash", trash)

In [1327]:
love = []
for i in range(24783):
  count, love = countOccurrences(classdf.tweet[i],"love", love)

In [1328]:
real = []
for i in range(24783):
  count, real = countOccurrences(classdf.tweet[i],"real", real)

In [1329]:
good = []
for i in range(24783):
  count, good = countOccurrences(classdf.tweet[i],"good", good)

In [1330]:
black = []
for i in range(24783):
  count, black = countOccurrences(classdf.tweet[i],"black", black)

In [1331]:

encodedHateDF = pd.DataFrame(faggot)

In [1332]:
encodedHateDF

,0
0,0
1,0
2,0
3,0
4,0
...,...
24778,0
24779,0
24780,0
24781,0


In [1333]:
encodedHateDF.rename(columns = {0: "Faggot"}, inplace = True)

In [1334]:
#encodedHateDF["Bitch"] = bitch

In [1335]:
#encodedHateDF["Fuck"] = fuck

In [1336]:
#encodedHateDF["Nigga"] = nigga

In [1337]:
#encodedHateDF["Ass"] = ass

In [1338]:
encodedHateDF["Fag"] = fag

In [1339]:
#encodedHateDF["Niggas"] = niggas

In [1340]:
encodedHateDF["Fucking"] = fucking

In [1341]:
encodedHateDF["Faggots"] = faggots

In [1342]:
encodedHateDF["White"] = white

In [1343]:
encodedHateDF["Trash"] = trash

In [1344]:
encodedHateDF["Love"] = love

In [1345]:
encodedHateDF["Real"] = real

In [1346]:
encodedHateDF["Good"] = good

In [1347]:
encodedHateDF["Black"] = black

In [1348]:
encodedHateDF

,Faggot,Fag,Fucking,Faggots,White,Trash,Love,Real,Good,Black
0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
24778,0,0,0,0,0,1,0,0,0,0
24779,0,0,0,0,0,0,0,0,0,0
24780,0,0,0,0,0,0,0,0,0,0
24781,0,0,0,0,0,0,0,0,0,0


In [1349]:
wordDF = pd.merge(encodedDf, encodedHateDF, left_index=True, right_index=True)

In [1350]:
wordDF.head()

,Bitch,Bitches,Hoes,Pussy,Hoe,Fuck,Ass,Nigga,Shit,Niggas,...,Faggot,Fag,Fucking,Faggots,White,Trash,Love,Real,Good,Black
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [1351]:
target = classdf["class"]

In [1352]:
sm = SMOTE()
X,Y = sm.fit_resample(wordDF,target)

In [1353]:
x_train,x_test,x_target_train,x_target_test = train_test_split(X,Y,test_size=0.2,shuffle=True,random_state=0)

In [1354]:
#rfc=RandomForestClassifier(n_estimators= 250, max_depth=100)
#dtc=DecisionTreeClassifier()
#lr=LogisticRegression(max_iter=600)
#knnc=KNeighborsClassifier()
#svc=SVC()
#nb=GaussianNB()

In [1355]:
#models = [("rfc",rfc),("dtc",dtc),("lr",lr),("knnc",knnc),("svc",svc),("nb",nb)]

In [1356]:
#voting = VotingClassifier(models)

In [1357]:
#voting.fit(x_train,x_target_train)

In [1358]:
#x_pred = voting.predict(x_test)

In [1359]:
#accuracy_score(x_target_test,x_pred)

In [1360]:
model = xgb.XGBClassifier()
model.fit(x_train,x_target_train)

XGBClassifier(objective='multi:softprob')

In [1361]:
x_pred = model.predict(x_test)

In [1362]:
accuracy_score(x_target_test,x_pred)

0.6961959353830119

In [1363]:
model = gbm.LGBMClassifier()

In [1364]:
model.fit(x_train,x_target_train)

LGBMClassifier()

In [1365]:
x_pred = model.predict(x_test)

In [1366]:
accuracy_score(x_target_test,x_pred)

0.7023623414973076